In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('scholarship_data.csv')

In [3]:
df.head(3)

,Name,Auto number,Program check,Program name,Type of Opportunity,Annual scholarship?,Study level,Host Country,Discipline,Value,Note,Link,Deadline?,Date updated,Checked?
0,NaN,357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1439,NaN,NaN,NaN,NaN,NaN,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aarhus BS,1231,NaN,Aarhus BS,University-sponsor Scholarship,checked,Master,Denmark,Business and finance,"less than 50% tuition fee ,100% tuition fee",NaN,http://bss.au.dk/en/programmes/master/,October,20/9/2017,NaN


In [4]:
df.columns.values

array(['Name', 'Auto number', 'Program check', 'Program name',
       'Type of Opportunity', 'Annual scholarship?', 'Study level',
       'Host Country', 'Discipline', 'Value', 'Note', 'Link', 'Deadline?',
       'Date updated', 'Checked?'], dtype=object)

In [5]:
df.drop(labels=['Auto number', 'Program check','Program name','Annual scholarship?','Date updated','Checked?'],axis=1,inplace=True)
df.drop(labels=df[df.Name.isnull() & df.Link.isnull()].index,inplace=True)

In [6]:
df.drop(labels=df[df['Deadline?'].isna()].index,inplace=True)

In [7]:
df.reset_index(drop=True,inplace=True)
df.head(3)

,Name,Type of Opportunity,Study level,Host Country,Discipline,Value,Note,Link,Deadline?
0,Aarhus BS,University-sponsor Scholarship,Master,Denmark,Business and finance,"less than 50% tuition fee ,100% tuition fee",NaN,http://bss.au.dk/en/programmes/master/,October
1,Amandus H. Lundqvist Scholarship Program,University-sponsor Scholarship,Master,Netherland,A LOT OF DISCIPLINES,Full scholarship,Continuation of the scholarship during the two...,https://www.tue.nl/en/studeren/studying-at-tue...,Febuary
2,Art Major Asian(AMA) Scholarship,University-sponsor Scholarship,"Undergraduate ,Master",South Korea,"""Arts, Design and Photography""",Full scholarship,Ứng viên cần có tài năng về nghệ thuật vượt trội,http://karts.ac.kr:8090/?MID=htmlContent&IDX=375,June


In [8]:
df['scholarship_id'] = df.index + 1

In [9]:
df['Deadline?'].str.split(',')

0               [October]
1               [Febuary]
2                  [June]
3               [January]
4               [October]
              ...        
1180    [April, November]
1181              [April]
1182            [October]
1183      [May, December]
1184           [November]
Name: Deadline?, Length: 1185, dtype: object

In [10]:
def clean_by_field(field_name):
    field = df[field_name].str.split(',')
    unique_values = []
    for records in field:
        try:
            unique_values += [i.strip(' ') for i in records]
        except:
            print('s')
    unique_values = set(unique_values)
    unique_values
    field_df = pd.DataFrame(data=df[field_name], columns=[field_name])
    field_df.reset_index()
    field_df['scholarship_id'] = field_df.index + 1
    lst = []
    for value in unique_values:
        matched_df = field_df[field_df[field_name].str.contains(value)][['scholarship_id',field_name]]
        matched_df[field_name] = value
        lst.append(matched_df)

    new_field_df = pd.concat(objs=lst, ignore_index=True)
    new_field_df['id'] = new_field_df.index + 1
    new_field_df = new_field_df[['id','scholarship_id', field_name]]

    return new_field_df

In [11]:
level_df = clean_by_field('Study level')
level_df.rename(mapper={'Study level': 'study_level'},axis=1,inplace=True)
level_df.loc[level_df.study_level == 'High school', 'study_level'] = 'Highschool'
level_df.loc[level_df.study_level == 'Post-graduate', 'study_level'] = 'Postgraduate'
level_df.to_csv('data\levels.csv',index=False)

In [12]:
discipline_df = clean_by_field('Discipline')
discipline_df.head()

discipline_df.rename(mapper={'Discipline':'discipline'},axis=1,inplace=True)

discipline_df.discipline.unique()

discipline_df.loc[discipline_df.discipline == 'A LOT OF DISCIPLINES', 'discipline'] = 'Multidisciplinary'
discipline_df.loc[discipline_df.discipline == '"Arts', 'discipline'] = 'Arts Design and Photography'
discipline_df.loc[discipline_df.discipline == 'Design and Photography"', 'discipline'] = 'Arts Design and Photography'
discipline_df.loc[discipline_df.discipline == '"Hospitality', 'discipline'] = 'Hospitality Tourism and Sports'
discipline_df.loc[discipline_df.discipline == 'Tourism and Sports"', 'discipline'] = 'Hospitality Tourism and Sports'
discipline_df.to_csv('disciplines.csv', index=False)
discipline_df.discipline.unique()
deadline_df = clean_by_field('Deadline?')
deadline_df.rename(mapper={'Deadline?':'deadline'},axis=1,inplace=True)
deadline_df.to_csv('data\deadlines.csv',index=False)

In [13]:
value_df = clean_by_field('Value')

C:\Users\quanc\AppData\Local\Temp\ipykernel_9824\985428034.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matched_df = field_df[field_df[field_name].str.contains(value)][['scholarship_id',field_name]]


In [14]:
value_df.head(3)
value_df.rename(mapper={'Value':'value'},axis=1,inplace=True)
value_df.value.unique()
value_df.loc[value_df.value == 'Full scholarship','value'] = 'Fullride'
value_df.loc[value_df.value == '100% tuition fee','value'] = 'Full tuition'
value_df.loc[value_df.value == '50% tuition fee','value'] = 'Half tuition'
value_df.loc[value_df.value == 'Cash award','value'] = 'Cash'
value_df.loc[value_df.value == 'more than 50% tuition fee','value'] = 'More than half'
value_df.loc[value_df.value == 'less than 50% tuition fee','value'] = 'Less than half'
value_df.to_csv('data\\values.csv',index=False)

In [15]:
df.head(1)

,Name,Type of Opportunity,Study level,Host Country,Discipline,Value,Note,Link,Deadline?,scholarship_id
0,Aarhus BS,University-sponsor Scholarship,Master,Denmark,Business and finance,"less than 50% tuition fee ,100% tuition fee",NaN,http://bss.au.dk/en/programmes/master/,October,1


In [16]:
scholarship_df = df[['scholarship_id','Name','Type of Opportunity', 'Host Country', 'Link', 'Note']]
scholarship_df.head(2)
scholarship_df.rename(mapper={
    'Name':'name',
    'Type of Opportunity': 'sponsor',
    'Host Country': 'host',
    'Link': 'link',
    'Note': 'note'
},axis=1,inplace=True)
scholarship_df.loc[scholarship_df.sponsor == 'Conference Scholarship', 'sponsor'] = 'Conference'
scholarship_df.loc[scholarship_df.sponsor == 'Fellowship Scholarship', 'sponsor'] = 'Fellowship'
scholarship_df.loc[scholarship_df.sponsor == 'University-sponsor Scholarship', 'sponsor'] = 'University'
scholarship_df.loc[scholarship_df.sponsor == 'Exchange scholarship', 'sponsor'] = 'Exchange'
scholarship_df.loc[scholarship_df.sponsor == 'Government Scholarship', 'sponsor'] = 'Government'
scholarship_df.loc[scholarship_df.sponsor == 'Company-sponsor Scholarship', 'sponsor'] = 'Company'
scholarship_df.loc[scholarship_df.sponsor == 'No specified', 'sponsor'] = 'N/A'
scholarship_df.head(2)
scholarship_df.to_csv('data\scholarships.csv',index=False)

C:\Users\quanc\AppData\Local\Temp\ipykernel_9824\3312111771.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scholarship_df.rename(mapper={


In [17]:
scholarship_df.sponsor.unique()

array(['University', 'Exchange', 'Government', 'Fellowship', 'Conference',
       'Company', 'N/A'], dtype=object)